<a href="https://colab.research.google.com/github/wdavjr/wd-mta-major-incidents-analysis/blob/main/mta_major_incidents_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [25]:
incidents = pd.read_csv('https://raw.githubusercontent.com/wdavjr/wd-mta-major-incidents-analysis/refs/heads/main/MTA_Subway_Major_Incidents__Beginning_2020.csv')

In [26]:
incidents.head()

,month,division,line,day_type,category,count
0,2020-01-01,A DIVISION,1,1,Other,1
1,2020-01-01,A DIVISION,2,1,Persons on Trackbed/Police/Medical,1
2,2020-01-01,A DIVISION,2,2,Other,1
3,2020-01-01,A DIVISION,3,1,Persons on Trackbed/Police/Medical,1
4,2020-01-01,A DIVISION,3,1,Signals,2


In [23]:
incidents.groupby("day_type")['count'].sum()

,count
day_type,
1,2288
2,279
